In [3]:
#;.pykx.disableJupyter()

PyKX now running in 'python' mode (default). All cells by default will be run as python code. 
Include '%%q' at the beginning of each cell to run as q code. 


In [4]:
# https://code.kx.com/pykx/3.0/examples/jupyter-integration.html#q-first-mode
import pykx as kx
kx.util.jupyter_qfirst_enable()

PyKX now running in 'jupyter_qfirst' mode. All cells by default will be run as q code. 
Include '%%py' at the beginning of each cell to run as python code. 


##### Initialization

In [5]:
example:([]a:1 2 3;b: `mini`example`table);
keyedExample:([b:`mini`keyed`example]a: 1 2 3);
complexTab:([]c:(1 2;3 4;5 6);d: `more`complex`table)

# Tables


**Learning Outcomes**

To understand: 
* How to investigate and identify a table
* The two types of table
* How to create a table
* Accessing tables
* How to insert data into a table

# Introduction

Unsurprisingly, [tables](https://code.kx.com/q4m3/8_Tables/) form the core of Kx technology and much of what we've learned up to now has been building up to tables. Tables are the **most important** data structures in kdb+ and is where the vast majority of all data in kdb+/q is stored.


## Table discovery 
Firstly let's look at what tables we have in our workspace:

In [6]:
tables[]               //typical usage - default to passing `.
tables[`.]~tables[]

`complexTab`example`keyedExample
1b


We can see how many records our tables have using count: 

In [7]:
count example 
count complexTab

3
3


In [8]:
\a

`complexTab`example`keyedExample


To find out the name of the columns of a table, we can apply the keyword [cols](https://code.kx.com/q/ref/cols):

In [9]:
cols example           //returns a list of symbols

`a`b


If we want to return the entire table contents we can do so by calling the table by name.

In [10]:
example 

a b      
---------
1 mini   
2 example
3 table  


In [11]:
complexTab     //This table is more complex, the first column c is actually a list of lists!

c   d      
-----------
1 2 more   
3 4 complex
5 6 table  


## Table schema
Perhaps the most important information associated with any table is the table metadata - obtained by using the keyword [`meta`](https://code.kx.com/v2/ref/meta).

The `meta` command takes a table name as input, and will return a table of available columns and their types and other information. The following columns are produced:

- `c`: column name
- `t`: column [type](https://code.kx.com/v2/ref/#datatypes)
- `f`: [foreign keys](https://code.kx.com/q4m3/8_Tables/#85-foreign-keys-and-virtual-columns)
- `a`: [attributes](https://code.kx.com/v2/basics/syntax/#attributes) - any column modifiers applied for performance characteristics 


In [12]:
meta example 

c| t f a
-| -----
a| j    
b| s    


The first column `t` includes the type information specified by the char value of the type. If this data is itself comprised of lists of a particular type, this char value will be capitalized - we can see this by looking at our complex table: 

In [13]:
meta complexTab

c| t f a
-| -----
c| J    
d| s    


##### Exercise 

Inspect the table definition and types for the `keyedExample` table 

In [ ]:
meta keyedExample

In [16]:
meta keyedExample
keyedExample

c| t f a
-| -----
b| s    
a| j    
b      | a
-------| -
mini   | 1
keyed  | 2
example| 3


# Types of Tables

There are two types of tables in kdb+/q: 
* Unkeyed tables - simple tables which are just lists of records
* Keyed tables - these are really dictionaries (which present as tables) where we associate a table of keys with a table of records.

## How to identify whether a table is keyed or unkeyed
There are two differences that we can use to distinguish - the output in the console and the type. 

In [17]:
example 
keyedExample //can see vertical line

a b      
---------
1 mini   
2 example
3 table  
b      | a
-------| -
mini   | 1
keyed  | 2
example| 3


In [18]:
type example 
type keyedExample //this is the type of a dictionary

98h
99h


## Unkeyed tables


An unkeyed table is:

* A list of dictionaries - each dictionary is a record in our table
* A flipped (transposed) column dictionary 
* A collection of lists of equal length (called columns) - the values in our column dictionary

###  A table is a list of dictionaries

An unkeyed table can be directly queried/modified as if it were a list of dictionaries. 

In [20]:
example             //our table as defined
count example       //the number of records in our table

a b      
---------
1 mini   
2 example
3 table  
3


In [21]:
first example           //the first record in our table - it's a dictionary! 

a| 1
b| `mini


##### Exercise

Return the final row from our `example` table and verify it is in fact a dictionary.

In [ ]:
last example
type last example   //it's a dictionary! 

//for unkeyed tables we can use indexing too!
example 2

In [24]:
last example
type last example

a| 3
b| `table
99h


### A table is a flipped column dictionary

Since kdb+/q works in vector based fashion, each column in the table is treated as a large vector which is associated with that column name. 

In [25]:
show d:flip example     //d is a column dictionary i.e. values are lists of equal length
                           //and the dictionary keys are symbols  

a| 1    2       3    
b| mini example table


This new variable `d` is just a standard dictionary;

In [26]:
d`b               //We can index keys like any dictionary
d[`b;0 1]         //Index at depth

`mini`example`table
`mini`example


In [27]:
d[`a;0 1]*:2;      //Amend items
d                  //Notice the change

a| 2    4       3    
b| mini example table


In [28]:
flip d             //We can use flip to turn this back to a table

a b      
---------
2 mini   
4 example
3 table  


### A table is a collection of lists of equal length (called columns)

We have mentioned that kdb+/q treats each column within a table as a vector and to form a table we require each of the columns to be an equal length.  

In [29]:
example `a          //We can extract a column list with lookup notation similar to dictionaries - example[`a]
example`a`b         //We can extract multiple columns

1 2 3
1    2       3    
mini example table


##### Exercise

Create a new column `c:3.14 2.72 299792458` in our `example` table, using dictionary assignment syntax.

In [ ]:
example[`c]: 3.14 2.72 299792458  //creating a new column and assigning 
example

In [31]:
example[`c]:3.14 2.72 299792458
example 

a b       c           
----------------------
1 mini    3.14        
2 example 2.72        
3 table   2.997925e+08


## Keyed tables

A keyed table is a special form of a dictionary where the key and value are both tables. We can extract the keys and values using the inbuilt operators, [key](https://code.kx.com/q/ref/key/) and [value](https://code.kx.com/q/ref/value/), just like we did with dictionaries. 

In [32]:
keyedExample 

b      | a
-------| -
mini   | 1
keyed  | 2
example| 3


In [33]:
key keyedExample 

b      
-------
mini   
keyed  
example


In [34]:
value keyedExample

a
-
1
2
3


We can actually make a keyed table (table to table dictionary) explicitly using the `!` keyword with two unkeyed tables of equal length:

In [35]:
example!complexTab //result is a keyed table 

a b       c           | c   d      
----------------------| -----------
1 mini    3.14        | 1 2 more   
2 example 2.72        | 3 4 complex
3 table   2.997925e+08| 5 6 table  


### Differences from unkeyed tables

With keyed tables, we will see that the operations we tried for unkeyed tables will return different values: 

In [36]:
keyedExample
first keyedExample //returns dictionary association of our keyed tables "value" table
first value keyedExample
first[keyedExample]~first value keyedExample

b      | a
-------| -
mini   | 1
keyed  | 2
example| 3
a| 1
a| 1
1b


We can also see that attempting to retrieve the columns by name as we did with unkeyed tables does not work either: 

In [38]:
keyedExample[`b]

a| 


When we attempt a dictionary lookup in our table, it is performed with reference to the dictionary key and like standard dictionaries, when the value is not found a null value is returned. 

In [53]:
key keyedExample      //what are the keys of our table
keyedExample[`mini]   //using a key value to return the associated "value table" values 
keyedExample[`large]  //using a key that does not exist

/ kd:value keyedExample
/ kd[`c]:3.14 2.72 299792458
/ (key keyedExample)!kd

keyedExample2:update c:3.14 2.72 299792458 from keyedExample
keyedExample2
keyedExample2[`mini]
keyedExample2[`dontexist]

b      
-------
mini   
keyed  
example
a| 1
a| 
b      | a c           
-------| --------------
mini   | 1 3.14        
keyed  | 2 2.72        
example| 3 2.997925e+08
a| 1
c| 3.14
a| 0N
c| 0n


When we pass an associated key value our data is returned - further detail in later sections.

## Keying and unkeying a table 


It is possible to convert dynamically between an unkeyed table having a column of potential key values and the corresponding keyed table using the keyword [xkey](https://code.kx.com/q/ref/keys/).

In [54]:
`a xkey example    

a| b       c           
-| --------------------
1| mini    3.14        
2| example 2.72        
3| table   2.997925e+08


We can use this with our already keyed table also:

In [55]:
keyedExample
newKeyed:`a xkey keyedExample   //changing our key to be the column `a instead 
newKeyed

b      | a
-------| -
mini   | 1
keyed  | 2
example| 3
a| b      
-| -------
1| mini   
2| keyed  
3| example


In [56]:
newKeyed[3]    //Now the key has changed, we use our a values for retrieval

b| example


##### Exercise 
The below will throw an error - do you know why? 

In [57]:
newKeyed[`mini]            

QError: type

The reason for the error is that our key is no longer the value for the `b` column as we used before. Since our new key is `a` and this column has a type of long, passing a symbol value for retrieval will throw a `'type` error. If in doubt, checking the `meta` for types is very helpful.

To convert a keyed table back to a regular unkeyed table, we can also use `xkey` with an empty general list as the left operand.

In [58]:
() xkey keyedExample

b       a
---------
mini    1
keyed   2
example 3


 ##### The Bang (`!`) operator 
There is another operator we can use to change the keys on a table - there is an overload of the `!` operator referred to as [unkey](https://code.kx.com/q/ref/enkey/#unkey) which will remove or add keys to a table.

In [59]:
0!keyedExample        //Here the 0 means use 0 of the columns to key the table

b       a
---------
mini    1
keyed   2
example 3


##### Exercise 

Create a table called `bigExample` which combines the `example` and `complexTab` table to have the column order <code>`c`d`a`b c</code>. This table should be unkeyed. 

Note: there are many different ways of doing this

In [ ]:
show bigExample: 0!complexTab!example

In [61]:
example
complexTab

0!complexTab!example

a b       c           
----------------------
1 mini    3.14        
2 example 2.72        
3 table   2.997925e+08
c   d      
-----------
1 2 more   
3 4 complex
5 6 table  
c   d       a b       c           
----------------------------------
1 2 more    1 mini    3.14        
3 4 complex 2 example 2.72        
5 6 table   3 table   2.997925e+08


# Creating tables
The syntax to create a table is as follows:

` ([kcolname1:klist1;...;kcolnameM:klistM] colname1:list1;...;colnameN:listN) `

where:
+ `kcolname1` ... `kcolnameM` are the names of the key columns
+ `klist1` ... `klistM` are the lists of key values associated with the respective key column
+ `colname1` ... `colnameN` are the column names 
+ `list1` ... `listN` are the lists of values associated with the respective column and all lists are of equal length.

## Unkeyed Tables
For now, let's say that an unkeyed table is a table that doesn't have a primary key associated with it.


### Creating a table with existing data  

In [62]:
show trade:([]sym:`JPM`IBM`BP;size:100 25 54;price:3.45 5.21 6.33)  

sym size price
--------------
JPM 100  3.45 
IBM 25   5.21 
BP  54   6.33 


If there is a mix of atomic and list values kdb+/q will handle this in the same way it handles operations like addition and will fill the column accordingly with the atomic value:  

In [63]:
show trade:([]sym:`JPM`IBM`BP;size:100 25 54;price:3.45 5.21 6.33;ex:`NYSE)

sym size price ex  
-------------------
JPM 100  3.45  NYSE
IBM 25   5.21  NYSE
BP  54   6.33  NYSE


However, we can only do this when not all the values are atomic - if they all are it will throw an error as the table definition syntax requires lists in at least some columns

In [64]:
([]sym:`JPM;size:100;price:3.45) //error

QError: rank

In [65]:
([]sym:enlist[`JPM];size:enlist[100];price:enlist[3.45])  //if we make our atoms lists we can define a single row

sym size price
--------------
JPM 100  3.45 


### Creating an empty table
Below is a simple definition of an empty table with column names and empty lists. 

In [66]:
show emptyTrade:([]sym:();size:();price:())
// horizontal lines are the signature in the display of a table

sym size price
--------------


In [67]:
meta emptyTrade    //we have no type information as we only used general lists

c    | t f a
-----| -----
sym  |      
size |      
price|      


We can also specify the type that we want each list to be associated with by typing the lists when defining:

In [68]:
show emptyTradeTyped:([]sym:`$();size:`long$();price:`float$())
meta emptyTradeTyped

c    | t f a
-----| -----
sym  | s    
size | j    
price| f    
sym size price
--------------


## Keyed Tables

When defining a table, placing column(s) in the square brackets indicates a key:

In [69]:
show trade:([]sym:`JPM`IBM`BP;size:100 25 54;price:3.45 5.21 6.33)   // unkeyed table

sym size price
--------------
JPM 100  3.45 
IBM 25   5.21 
BP  54   6.33 


In [70]:
show tradeKeyed:([sym:`JPM`IBM`BP]size:100 25 54;price:3.45 5.21 6.33)   // table keyed on sym

sym| size price
---| ----------
JPM| 100  3.45 
IBM| 25   5.21 
BP | 54   6.33 


##### Exercise 
Create a table `quote` that looks like this:

|Time|sym|bid|ask|
|---|---|---|---|
|2020.04.16D09:30:00.0000|GE|100|25|
|2020.04.16D09:31:00.0000|GE|100|25|

Create another table `quoteKey` that is the same as the above except it is keyed on `sym`. 

Do so by: 
*  Explicit table definition 
*  Modification of the `quote` table



In [ ]:
show quote:([]time:(2020.04.16D09:30:00.0000;2020.04.16D09:31:00.0000);sym:`GE`GE;bid:(100;100);ask:(25;25))

In [ ]:
show quoteKey:([sym:`GE`GE];time:(2020.04.16D09:30:00.0000;2020.04.16D09:31:00.0000);bid:(100;100);ask:(25;25))

In [ ]:
//modification of the quote table 
`sym xkey quote

In [77]:
quote:([] Time:2020.04.16D09:30:00.0000 2020.04.16D09:31:00.0000; sym:`GE; bid:100; ask:25)
quoteKey:([sym:2#`GE] Time:2020.04.16D09:30:00.0000 2020.04.16D09:31:00.0000; bid:100; ask:25)
quoteKey // 1st way

`sym xkey quote // 2nd way

sym| Time                          bid ask
---| -------------------------------------
GE | 2020.04.16D09:30:00.000000000 100 25 
GE | 2020.04.16D09:31:00.000000000 100 25 
sym| Time                          bid ask
---| -------------------------------------
GE | 2020.04.16D09:30:00.000000000 100 25 
GE | 2020.04.16D09:31:00.000000000 100 25 


# Accessing tables
We have outlined the differences between the two types of tables, due to their different structures accessing them will also be different. 

Let's break it down between accessing specific columns and rows.

## Accessing columns


### Unkeyed tables
There are two notations that allows us to retrieve a column from an unkeyed table: 
* Back-tick notation (like dictionary retrieval)
* Dot notation


In [78]:
trade[`sym] //retrieving the sym column using backtick notation 
trade.sym

`JPM`IBM`BP
`JPM`IBM`BP


We can access columns using the dot notation however it's important to know that this applies to the global table. The dot notation should be avoided particuarly within functions, unless you are absolutely clear on what you are doing. 

In [79]:
sum trade.size           //works fine

179


In [ ]:
getSumSizes:{sum x.size}  //write a function that does the same
getSumSizes trade         //q does not like this - it looks for a global x!

However, if we use the back-tick notation we don't encounter the same issue: 

In [80]:
getSumSizes:{sum x`size};   //changed to use ` notation
getSumSizes trade           //this works

179


##### Exercise 

Extract the columns bid and ask from our `quote` table: 

In [ ]:
quote
quote[`bid`ask]             //we can specify both together

In [81]:
quote[`bid`ask]

100 100
25  25 


### Keyed tables
As keyed tables are like dictionaries, we need to access them using the same methods as we used on dictionaries. 

In [82]:
tradeKey:([sym:`JPM`IBM`KX]size:10 20 30;price:1 2 3)
tradeKey

sym| size price
---| ----------
JPM| 10   1    
IBM| 20   2    
KX | 30   3    


In [83]:
key tradeKey  //will return the sym column as a table
(key tradeKey)[`sym] //indexing into the table to return the values of the column sym

sym
---
JPM
IBM
KX 
`JPM`IBM`KX


We can do the same operation on the value columns that are contained in the `tradeKey` table

In [84]:
(value tradeKey)[`size]
(value tradeKey)[`price]

10 20 30
1 2 3


<img src="../qbies.png" width="50px" style="width: 50px;padding-right:5px;padding-top:12px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> Keyed tables are accessed via their key, so  can't use the dot or back-tick notation like we did before with unkeyed tables. We will see the <code>exec</code> command in the Queries section will also allow us to access columns in a table.</i></p>



Another option would be to remove the key on the table and then access the column from the unkeyed table. Let's check out which method is more performant:

In [85]:
//unkey then access 
\ts:100000  (0! tradeKey)[`size]
//access via value table - the most space performant as we're subsetting the full table 
  //to just the value table in our keyed table
\ts:100000  (value tradeKey)[`size]
//access via exec - this is the most time performant 
\ts:100000 exec size from tradeKey    

65 1488
30 880
30 928


## Accessing rows

### Unkeyed tables

As unkeyed tables are lists of dictionaries, we can use list indexing to retrieve a particular row. 

In [86]:
trade
trade 1    //retrieving the second 

sym size price
--------------
JPM 100  3.45 
IBM 25   5.21 
BP  54   6.33 
sym  | `IBM
size | 25
price| 5.21


Similar to lists again, we can also use [#](https://code.kx.com/q/ref/take/) to get a subset of the rows:

In [87]:
3#1 2 3 4 5 //selecting the first 3 entries of the list
2#trade    //selecting the first 2 entries of the trade table
-2#trade    //selecting the last 2 entries of the trade table

1 2 3
sym size price
--------------
JPM 100  3.45 
IBM 25   5.21 
sym size price
--------------
IBM 25   5.21 
BP  54   6.33 


### Keyed table
Like all dictionaries, a keyed table is accessed by key:

In [88]:
tradeKey 
tradeKey`JPM       //retrieve the row at that key - here the key is like the index

sym| size price
---| ----------
JPM| 10   1    
IBM| 20   2    
KX | 30   3    
size | 10
price| 1


If we wanted to retrieve more than one value we can't operate in the same way as a standard dictionary and just provide a list of keys: 

In [89]:
tradeKey[`JPM`BP]  //try retrieve value for multiple keys

QError: length

If we want to return many values we need to supply our inputs like a table:

In [98]:
tradeKey[([]sym:`JPM`XXX`IBM)] //XXX doesn't exist in table therefore returning nulls for that sym

size price
----------
10   1    
          
40   10   


##### Exercise 

Update our `tradeKey` table to change the values for IBM to have a size of 40 and a price of 10.

In [ ]:
tradeKey[`IBM]: 40 10 //indexing into the row correponding to the IBM key 
                        //reassigning the values to 40 and 10
tradeKey

In [96]:
tradeKey[`IBM]:`size`price!(40;10)
tradeKey

sym| size price
---| ----------
JPM| 10   1    
IBM| 40   10   
KX | 30   3    


## Accessing a particular cell 

Putting this all together, we can continue to elide index into our tables to access particular cells. 

In [99]:
//unkeyed table 
trade
//keyed table 
tradeKey

sym size price
--------------
JPM 100  3.45 
IBM 25   5.21 
BP  54   6.33 
sym| size price
---| ----------
JPM| 10   1    
IBM| 40   10   
KX | 30   3    


Let's suppose in both instances we want to return the size values for JPM and IBM: 

In [100]:
//unkeyed table - first index by row, then by column
trade[0 1;`size]
//OR - first index by column, then by row
trade[`size][0 1]  

//Keyed table - first index by key, then by column 
tradeKey[([]sym: `JPM`IBM);`size]

100 25
100 25
10 40


If we want to modify the size values for JPM and IBM to be 50 and 70 respectively we can do so via reassignment: 

In [101]:
//unkeyed 
trade[0 1;`size]: 50 70 
trade 

sym size price
--------------
JPM 50   3.45 
IBM 70   5.21 
BP  54   6.33 


In [102]:
//keyed 
tradeKey[([]sym: `JPM`IBM);`size]: 50 70 
tradeKey

sym| size price
---| ----------
JPM| 50   1    
IBM| 70   10   
KX | 30   3    


##### Exercise

Amend the second row of our `example` table to change our column value for `b` to `exercise` and for `a` to 15.

In [ ]:
example 

//we can do this individually: 
example[1;`a]: 15
example[1;`b]: `exercise

//or we can do them both together 
example[1;`a`b]:(15;`exercise)   

//either method is fine!
example

In [106]:
example[1;`b`a]: (`exercise;15)
example

a  b        c           
------------------------
1  mini     3.14        
15 exercise 2.72        
3  table    2.997925e+08


# Inserting Data 
There are many ways to insert data into a table. The two simplest methods involve using the built-in functions
* [insert](https://code.kx.com/q/ref/insert/) 
* [upsert](https://code.kx.com/q/ref/upsert/)

## Insert
The insert function is used to append data to a table. The syntax is:

    `tableName insert data

<img src="../qbies.png" width="50px" style="width: 50px;padding-right:5px;padding-top:2px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i>Insert <b>requires</b> persistence of the inserted data, otherwise it will error - hence the use of the <code>`</code> to amend the table globally by reference. </i></p>

### Unkeyed tables
Let's take an empty table for this example and try to append some data to it using `insert`:

In [107]:
show trade:([]sym:`$();size:`long$();price:`float$()) // Creating an empty table
meta trade

c    | t f a
-----| -----
sym  | s    
size | j    
price| f    
sym size price
--------------


In [108]:
`trade insert(`BP;100;12.44) 
//insert returns the index/indices that the data has been inserted into

,0


In [109]:
trade insert (`BP;100;12.44) 
//we get a type error if we try to apply the insert without the global reference

QError: type

##### Exercise

Insert a row into your `quote` table. 
* time:the timestamp right now
* sym:`JPM
* bid:120
* ask:60

In [ ]:
quote 
`quote insert (.z.P;`JPM;120;60)
quote

In [111]:
quote
`quote insert (.z.p;`JPM;120;60)
quote

Time                          sym bid ask
-----------------------------------------
2020.04.16D09:30:00.000000000 GE  100 25 
2020.04.16D09:31:00.000000000 GE  100 25 
,2
Time                          sym bid ask
-----------------------------------------
2020.04.16D09:30:00.000000000 GE  100 25 
2020.04.16D09:31:00.000000000 GE  100 25 
2025.02.27D17:53:22.050791980 JPM 120 60 


We can also use dictionaries with insert:

In [112]:
`trade insert `price`sym`size!(12.45;`MSFT;400) //inserting one row using a dictionary
trade

,1
sym  size price
---------------
BP   100  12.44
MSFT 400  12.45


### Keyed tables

We can use insert in the same way as we have done above with unkeyed tables.

In [113]:
show tradeKey:([sym:`$()]size:`long$();price:`float$()) //defining table
`tradeKey insert(`BP;100;4.33)                          //works fine
tradeKey                                                //our row has been added

,0
sym| size price
---| ----------
BP | 100  4.33 
sym| size price
---| ----------


In [114]:
`tradeKey insert `sym`price`size!(`KX;4.55;120)         //using a dictionary instead
tradeKey 

,1
sym| size price
---| ----------
BP | 100  4.33 
KX | 120  4.55 


However, this will throw an error if a record of that key already exists in the table:

In [115]:
`tradeKey insert(`BP;200;5.33)                           //insert error

QError: insert

### Bulk inserts

In the example above, we have inserted one entry into the trade table. The beauty of `insert` is that we can also perform bulk inserts:

In [116]:
// inserting two rows using a list of lists
`trade insert(`IBM`AAPL;200 300;15.53 14.39)
trade 

2 3
sym  size price
---------------
BP   100  12.44
MSFT 400  12.45
IBM  200  15.53
AAPL 300  14.39


A drawback when using dictionaries is we can't bulk insert in the same way as we saw with the lists: 

In [117]:
//inserting two rows using a dictionary
`trade insert `sym`size`price!((`MSFT;`JPM);(400;200);(12.45;11.5)) 

QError: type

We can get around that with relative ease by valuing our dictionary: 

In [118]:
//keys in the dict need to match order of columns in the table
`trade insert value `sym`size`price!((`MSFT;`JPM);(400;200);(12.45;11.5)) 

4 5


Finally, we can also use `insert` with another table with a consistent schema to append the table data to our target table: 

In [119]:
//flipping our dictionary to a table
show tab:flip `sym`size`price!((`KX;`KX);(400;200);(12.45;11.5))
`trade insert tab 

6 7
sym size price
--------------
KX  400  12.45
KX  200  11.5 


In [120]:
trade

sym  size price
---------------
BP   100  12.44
MSFT 400  12.45
IBM  200  15.53
AAPL 300  14.39
MSFT 400  12.45
JPM  200  11.5 
KX   400  12.45
KX   200  11.5 


In [121]:
`size`price`sym#tab                //reordering our tab table 
`trade insert `size`price`sym#tab  //we can insert this too 
trade 

size price sym
--------------
400  12.45 KX 
200  11.5  KX 
8 9
sym  size price
---------------
BP   100  12.44
MSFT 400  12.45
IBM  200  15.53
AAPL 300  14.39
MSFT 400  12.45
JPM  200  11.5 
KX   400  12.45
KX   200  11.5 
KX   400  12.45
KX   200  11.5 


By passing tables we can get the benefit of a bulk `insert`, while also getting the benefit of not having to ensure correct ordering on our data columns. In the event of missing columns, null data is used:

In [122]:
`sym`size#tab 
`trade insert `sym`size#tab
trade

sym size
--------
KX  400 
KX  200 
10 11
sym  size price
---------------
BP   100  12.44
MSFT 400  12.45
IBM  200  15.53
AAPL 300  14.39
MSFT 400  12.45
JPM  200  11.5 
KX   400  12.45
KX   200  11.5 
KX   400  12.45
KX   200  11.5 
KX   400       
KX   200       


This also applies to keyed tables, so long as we don't use any keys that are already present in the table: 

In [123]:
`tradeKey insert flip `sym`size`price!(`FD`CITI;102 10;23.9 211.1)
tradeKey

2 3
sym | size price
----| ----------
BP  | 100  4.33 
KX  | 120  4.55 
FD  | 102  23.9 
CITI| 10   211.1


##### Exercise

Given a `trade` table, use `insert` to add two new rows with the following values: 

```sym:`GE`MSFT,size:40 60,price:8.12 10.36```

In [ ]:
`trade insert (`GE`MSFT;40 60;8.12 10.36) //using a list of lists
//or using a dictionary 
// `trade insert value `sym`size`price!((`GE;`MSFT);(40;60);(8.12;10.36))

In [124]:
`trade insert flip `sym`size`price!(`GE`MSFT;40 60;8.12 10.36)

12 13


## Upsert 

The upsert template is like `insert`, only `upsert` will work on a table passed either by value or by reference and so is more flexible than `insert`. The syntax is the same as insert:

<code>\`tableName upsert data</code> 

### Unkeyed tables
All the examples for insert all work the same for `upsert`.

In [125]:
show trade:([]sym:`$();size:`long$();price:`float$())
`trade upsert (`GOOG;230;15.42)            //upserting one row
`trade upsert `sym`size`price!(`JPM;300;20.31) //upserting a dictionary 
trade 

trade
trade
sym  size price
---------------
GOOG 230  15.42
JPM  300  20.31
sym size price
--------------


When using `upsert` we can both pass-by-name and pass-by-value:

In [126]:
trade upsert (`GS;340;30.4)  //we are returned the table with the new data 
trade                        //we can see the change isn't persisted however 

sym  size price
---------------
GOOG 230  15.42
JPM  300  20.31
GS   340  30.4 
sym  size price
---------------
GOOG 230  15.42
JPM  300  20.31


### Keyed tables
With a keyed table, `upsert` works in the same way, however, the `upsert` function will **overwrite the key** if it's 
already present rather than throwing an error. 


In [127]:
tradeKey       //our table from earlier

sym | size price
----| ----------
BP  | 100  4.33 
KX  | 120  4.55 
FD  | 102  23.9 
CITI| 10   211.1


In [128]:
//upserting by reference 
`tradeKey upsert(`BP;200;5.33)    //upsert returns the table name, not the indices
tradeKey                          //row at that key has been updated

tradeKey
sym | size price
----| ----------
BP  | 200  5.33 
KX  | 120  4.55 
FD  | 102  23.9 
CITI| 10   211.1


A benefit of the pass-by-value effect of `upsert` is we can add new data to our table and assign to a new value while leaving our original table untouched:

In [129]:
show tradeKey2: tradeKey upsert(`FD;400;10.13)    //can add new key
tradeKey

sym | size price
----| ----------
BP  | 200  5.33 
KX  | 120  4.55 
FD  | 102  23.9 
CITI| 10   211.1
sym | size price
----| ----------
BP  | 200  5.33 
KX  | 120  4.55 
FD  | 400  10.13
CITI| 10   211.1


##### Exercise

Using the `quoteKey` table that we created in previous exercises, upsert a row with the current time, sym <code>\`AAPL</code>, bid of 60 and an ask of 200 to the table `quoteKey` and check to see if has been updated. 

Try upserting it again to see what happens.

In [ ]:
quoteKey

In [ ]:
`quoteKey upsert (`AAPL;.z.p;60;200)
quoteKey

In [ ]:
`quoteKey upsert (`AAPL;.z.p;60;200) //you will see that the time has changed
quoteKey

In [133]:
`quoteKey upsert flip `Time`sym`bid`ask!(enlist .z.p;`AAPL;enlist 60;enlist 200)
quoteKey

quoteKey
sym | Time                          bid ask
----| -------------------------------------
GE  | 2020.04.16D09:30:00.000000000 100 25 
GE  | 2020.04.16D09:31:00.000000000 100 25 
AAPL| 2025.02.27D17:59:48.291682989 60  200


### Bulk upserts 

Bulk additions to tables using `upsert` require data to be arranged row-by-row. 

In [134]:
`trade upsert ((`KX;240;16.71);(`FD;250;17.34)) //upserting bulk
//`trade insert (`KX`FD;240 250;16.71 17.34)     reminder - this was the insert syntax 

trade


In [135]:
trade //showing the new records inserted

sym  size price
---------------
GOOG 230  15.42
JPM  300  20.31
KX   240  16.71
FD   250  17.34


As with `insert` we can also use tables as bulk values to be added to our table, along with lists and dictionaries: 

In [136]:
trade upsert trade    //doubling up our trade table, but not presisting the change

sym  size price
---------------
GOOG 230  15.42
JPM  300  20.31
KX   240  16.71
FD   250  17.34
GOOG 230  15.42
JPM  300  20.31
KX   240  16.71
FD   250  17.34


On the plus side, this means that if we want to use a dictionary structure as a means of upserting data, we would need to flip the values to arrange them by row, or alternatively, flip the dictionary into a table: 

In [137]:
value `sym`size`price!(`JPM`MS;200 399; 10.2 39.5) 
flip value `sym`size`price!(`JPM`MS;200 399; 10.2 39.5) 
trade upsert flip value `sym`size`price!(`JPM`MS;200 399; 10.2 39.5)  //passing a dictionary
trade upsert flip `sym`size`price!(`JPM`MS;200 399; 10.2 39.5)        //passing a table

JPM  MS  
200  399 
10.2 39.5
`JPM 200 10.2
`MS  399 39.5
sym  size price
---------------
GOOG 230  15.42
JPM  300  20.31
KX   240  16.71
FD   250  17.34
JPM  200  10.2 
MS   399  39.5 
sym  size price
---------------
GOOG 230  15.42
JPM  300  20.31
KX   240  16.71
FD   250  17.34
JPM  200  10.2 
MS   399  39.5 


Again, missing columns will be handled with nulls: 

In [138]:
//passing a subsetted table
trade upsert flip `sym`size#`sym`size`price!(`JPM`MS;200 399; 10.2 39.5)  

sym  size price
---------------
GOOG 230  15.42
JPM  300  20.31
KX   240  16.71
FD   250  17.34
JPM  200       
MS   399       


##### Exercise 
Given a table defined as follows: 

``tradeEx:([]sym:`JPM`IBM`BP;size:100 25 54;price:3.45 5.21 6.33)``

Use `upsert` to add two new rows with the following values: 

```sym:`JPM`FD,size:50 50,price:4.76 3.21```

In [ ]:
show tradeEx:([]sym:`JPM`IBM`BP;size:100 25 54;price:3.45 5.21 6.33)  //defining our table

In [ ]:
tradeEx upsert ([] sym:`JPM`FD;size:50 50;price:4.76 3.21)  //bulk table upsert 
tradeEx upsert ((`JPM;50;4.76);(`FD;10;3.21))               //list of rows for upsert 

In [139]:
tradeEx:([]sym:`JPM`IBM`BP;size:100 25 54;price:3.45 5.21 6.33)

// upsert by value
tradeEx upsert flip `sym`size`price!(`JPM`FD;50 50; 4.76 3.21) // upserting a table
tradeEx upsert ([] sym:`JPM`FD;size:50 50;price:4.76 3.21) // upserting a table
tradeEx upsert flip value `sym`size`price!(`JPM`FD;50 50; 4.76 3.21) // upserting a dict
tradeEx upsert ((`JPM;50;4.76);(`FD;50;3.21)) // upserting row-by-row

// upsert by reference
`tradeEx upsert ([] sym:`JPM`FD;size:50 50;price:4.76 3.21) // upserting a table
tradeEx

sym size price
--------------
JPM 100  3.45 
IBM 25   5.21 
BP  54   6.33 
JPM 50   4.76 
FD  50   3.21 
sym size price
--------------
JPM 100  3.45 
IBM 25   5.21 
BP  54   6.33 
JPM 50   4.76 
FD  50   3.21 
sym size price
--------------
JPM 100  3.45 
IBM 25   5.21 
BP  54   6.33 
JPM 50   4.76 
FD  50   3.21 
sym size price
--------------
JPM 100  3.45 
IBM 25   5.21 
BP  54   6.33 
JPM 50   4.76 
FD  50   3.21 
tradeEx
sym size price
--------------
JPM 100  3.45 
IBM 25   5.21 
BP  54   6.33 
JPM 50   4.76 
FD  50   3.21 
